# **Oracle Benchmarking** : Empiric Network vs ChainLink

Focusing on LUNA crash period : May 6 to May 25

In [2]:
from dotenv import load_dotenv
load_dotenv()

import os
import json

from src.node import NodeRequester
from src.utils import get_selector_from_name

## Empiric Network Data Retrieval

We will use both StarkNet Sequencer & a synced Node running Pathfinder to retrieve data. Not that it is not yet "verifyable" since there's no P2P syncing between StarkNet nodes yet -> Therefore it's unprovable getting the correct timestamp nor the correct information of a TX.

In [6]:
sequencer_requester = NodeRequester(os.environ.get('STARKNET_SEQUENCER_URL'))
node_requester = NodeRequester(os.environ.get('STARKNET_NODE_URL'))